In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [55]:
matches = pd.read_csv("../data/raw/matches.csv")
matches.head()



,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [6]:
ml_df = matches.copy()

ml_df = ml_df[ml_df['winner'].notna()]
ml_df = ml_df[ml_df['result'] != 'no result']

ml_df = ml_df[['team1', 'team2', 'venue', 'toss_winner', 'toss_decision', 'season', 'winner']]
ml_df.head()


,team1,team2,venue,toss_winner,toss_decision,season,winner
0,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Royal Challengers Bangalore,field,2007/08,Kolkata Knight Riders
1,Kings XI Punjab,Chennai Super Kings,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,bat,2007/08,Chennai Super Kings
2,Delhi Daredevils,Rajasthan Royals,Feroz Shah Kotla,Rajasthan Royals,bat,2007/08,Delhi Daredevils
3,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium,Mumbai Indians,bat,2007/08,Royal Challengers Bangalore
4,Kolkata Knight Riders,Deccan Chargers,Eden Gardens,Deccan Chargers,bat,2007/08,Kolkata Knight Riders


In [ ]:
le_team = LabelEncoder()
le_venue = LabelEncoder()
le_toss_dec = LabelEncoder()
le_season = LabelEncoder()

ml_df['team1'] = le_team.fit_transform(ml_df['team1'])
ml_df['team2'] = le_team.transform(ml_df['team2'])
ml_df['toss_winner'] = le_team.transform(ml_df['toss_winner'])

ml_df['venue'] = le_venue.fit_transform(ml_df['venue'])
ml_df['toss_decision'] = le_toss_dec.fit_transform(ml_df['toss_decision'])
ml_df['season'] = le_season.fit_transform(ml_df['season'])
ml_df['winner'] = le_team.transform(ml_df['winner'])



In [11]:
ml_df['team1_is_home'] = (matches['team1'] == matches['city']).astype(int)
ml_df['team2_is_home'] = (matches['team2'] == matches['city']).astype(int)


In [12]:
ml_df['team1_won_toss'] = (matches['toss_winner'] == matches['team1']).astype(int)


In [19]:
matches = pd.read_csv("../data/raw/matches.csv")

# Convert "2007/08" → 2007
matches['season_year'] = matches['season'].apply(lambda x: int(str(x).split('/')[0]))


In [20]:
ml_df = matches.copy()

# remove no-winner cases
ml_df = ml_df[ml_df['winner'].notna()]
ml_df = ml_df[ml_df['result'] != 'no result']


In [21]:
ml_df['season_year'] = matches.loc[ml_df.index, 'season_year']


In [22]:
ml_df['is_modern_ipl'] = (ml_df['season_year'] >= 2018).astype(int)


In [23]:
ml_df['team1_toss'] = (matches.loc[ml_df.index, 'toss_winner'] == matches.loc[ml_df.index, 'team1']).astype(int)
ml_df['team2_toss'] = (matches.loc[ml_df.index, 'toss_winner'] == matches.loc[ml_df.index, 'team2']).astype(int)


In [24]:
ml_df['match_type'] = matches.loc[ml_df.index, 'match_type']


In [25]:
ml_df['match_type'] = LabelEncoder().fit_transform(ml_df['match_type'])


In [29]:
drop_cols = ['id', 'season', 'city', 'date']
ml_df = ml_df.drop(columns=drop_cols, errors='ignore')


In [39]:
from sklearn.preprocessing import LabelEncoder

le_winner = LabelEncoder()
ml_df['winner'] = le_winner.fit_transform(ml_df['winner'])


In [44]:
cols_to_remove = [
    'id', 'date', 'city', 'season',  # identifiers
    'player_of_match',               # not known pre-match
    'result', 'result_margin', 'target_runs', 'target_overs', 
    'super_over', 'method',          # leakage
    'umpire1', 'umpire2'             # no use in prediction
]

for col in cols_to_remove:
    if col in ml_df.columns:
        ml_df = ml_df.drop(columns=[col])


In [45]:
import joblib
joblib.dump(le_winner, "../models/winner_label_encoder.pkl")
print("Saved winner label encoder.")


Saved winner label encoder.


In [52]:
ml_df.to_csv("../data/processed/ml_data.csv", index=False)

In [ ]:
ml_df.dtypes.value_counts()
print("Object dtype columns:", ml_df.select_dtypes(include=['object']).columns.tolist())


Object dtype columns: []
